# Import Data

In [47]:
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics
import matplotlib.pyplot as plt
import sys
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


train_df = pd.read_csv('../Notebooks/kaggle/training_data.csv')
test_df = pd.read_csv('../Notebooks/kaggle/testing_data.csv')

# Split Data

In [48]:

def split_data(X, y, test_size):
    
    return train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

# Fit and Predict

In [49]:
def fit_and_predict(model, X_train, y_train, X_test):
    model = model.fit(X_train, y_train)
    prediction = model.predict(X_test)

    return prediction

# Calculate Statistics

In [50]:
def calculate_statistics(y_test, pred):
    acurancy = metrics.accuracy_score(y_test, pred)
    precision = metrics.precision_score(y_test, pred)
    recall = metrics.recall_score(y_test, pred)
    f1_score1 = metrics.f1_score(y_test, pred)
    aoc = metrics.roc_auc_score(y_test, pred)
    log_loss = metrics.log_loss(y_test, pred)
    
    print(f"Acurancy: {acurancy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1_score1}")
    print(f"AOC: {aoc}")
    print(f"Log Loss: {log_loss}")

# Confusion Matrix

In [51]:
def display_conf_matrix_graph(y_test, pred):
    conf_matrix = confusion_matrix(y_test, pred)
    
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = conf_matrix, display_labels = ["Not Paid", "Paid"])
    cm_display.plot()

# Decision Tree

In [52]:
features = [x for x in train_df.columns if x not in ['loan_status']]
X = train_df[features]
y = train_df['loan_status']

X_train, X_test, y_train, y_test = split_data(X, y, 0.3)

dTree = DecisionTreeClassifier()
dTree_pred = fit_and_predict(dTree, X_train, y_train, X_test)

calculate_statistics(y_test, dTree_pred)
display_conf_matrix_graph(y_test, dTree_pred)

# Aplication of a model (Decision Tree)

In [53]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

def random_forest():
    #train_df = pd.read_csv('../Notebooks/kaggle/final_data.csv')
    #test_df = pd.read_csv('../Notebooks/kaggle/final_data_kaggle.csv')
    #train_df.drop(columns=['loan_date','client_birth_number','client_gender','account_date','account_frequency','region'], inplace=True)
    #test_df.drop(columns=['loan_date','client_birth_number','client_gender','account_date','account_frequency','region'], inplace=True)
    
    #nan_rows = test_df[test_df.isnull().T.any()]
    features = [x for x in train_df.columns if x != 'loan_status']
    dev_X = train_df[features]
    dev_Y = train_df['loan_status']

    comp_X = test_df[features]
    loan_ids = test_df[['loan_id']]

    X_train,X_test,y_train,y_test=train_test_split(dev_X,dev_Y,test_size=0.2,random_state=100)

    #clf=RandomForestClassifier(n_estimators=100)

    clf = LogisticRegression()

    #clf = DecisionTreeClassifier()

    clf.fit(X_train,y_train)

    calculate_statistics(y_test, clf.predict(X_test))

    test_predict_probs = pd.DataFrame(clf.predict_proba(comp_X))
    loan_ids = loan_ids.assign(Predicted=test_predict_probs[0])
    loan_ids = loan_ids.rename(columns={'loan_id': 'Id'})
    #print(loan_ids)
    loan_ids.to_csv('results.csv', columns=['Id', 'Predicted'], index=False)
#random_forest()